In [2]:
pip install GoogleNews



   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- ------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\user\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [14]:
pip install huggingface_hub[hf_xet]


   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.8 MB 3.1 MB/s eta 0:00:01
   ---------------------- ----------------- 1.6/2.8 MB 2.8 MB/s eta 0:00:01
   --------------------------------- ------ 2.4/2.8 MB 3.2 MB/s eta 0:00:01
   --------------------------------- ------ 2.4/2.8 MB 3.2 MB/s eta 0:00:01
   ------------------------------------- -- 2.6/2.8 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\user\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [17]:
# install library kalau belum ada:
# pip install GoogleNews transformers torch openpyxl pandas

from GoogleNews import GoogleNews
from transformers import pipeline
import pandas as pd
from datetime import datetime, timedelta
import re

# ======================
# 0. Fungsi konversi tanggal ke dd/mm/yyyy
# ======================
def convert_to_date(date_str):
    """
    Mengubah string relative date GoogleNews ke format dd/mm/yyyy
    """
    today = datetime.today()
    date_str = (date_str or "").lower()  # aman jika None

    if "jam" in date_str:
        match = re.search(r'(\d+)\s*jam', date_str)
        hours = int(match.group(1)) if match else 1
        converted = today - timedelta(hours=hours)
    elif "hari" in date_str:
        match = re.search(r'(\d+)\s*hari', date_str)
        days = int(match.group(1)) if match else 1
        converted = today - timedelta(days=days)
    elif "minggu" in date_str:
        match = re.search(r'(\d+)\s*minggu', date_str)
        weeks = int(match.group(1)) if match else 1
        converted = today - timedelta(weeks=weeks)
    else:
        # Jika sudah berupa tanggal
        try:
            converted = datetime.strptime(date_str, "%d/%m/%Y")
        except:
            converted = today  # fallback jika tidak dikenali
    return converted.strftime("%d/%m/%Y")

# ======================
# 1. Daftar keyword pencarian
# ======================
keywords = [
    "Bank Riau Kepri Syariah",
    "BRK Syariah",
    "Bank Riau Kepri"
]

all_results = []

for keyword in keywords:
    googlenews = GoogleNews(lang="id", start="08/01/2025", end="08/31/2025")
    googlenews.search(keyword)
    results = googlenews.result()
    print(f"✅ Keyword '{keyword}' menghasilkan {len(results)} berita")
    all_results.extend(results)

# Hapus duplikasi berdasarkan judul
seen_titles = set()
unique_results = []
for r in all_results:
    if r.get("title") not in seen_titles:
        unique_results.append(r)
        seen_titles.add(r.get("title"))

print(f"\n📊 Total unik berita ditemukan: {len(unique_results)}\n")

if not unique_results:
    print("❌ Tidak ada berita ditemukan.")
    exit()

# ======================
# 2. Analisis Sentimen (pakai IndoBERT mdhugol)
# ======================
classifier = pipeline(
    "sentiment-analysis",
    model="mdhugol/indonesia-bert-sentiment-classification"
)

# Mapping label HuggingFace → Sentimen
label_map = {
    "LABEL_0": "Positif",
    "LABEL_1": "Netral",
    "LABEL_2": "Negatif"
}

berita_final = []

for news in unique_results:
    title = news.get("title", "")
    link = news.get("link", "")
    raw_date = news.get("date", "")
    date = convert_to_date(raw_date)  # <-- konversi tanggal

    try:
        result = classifier(title)[0]
        label = label_map.get(result["label"], "Netral")
        score = round(result["score"], 2)

        # Aturan tambahan (agar Netral bisa lebih jelas)
        if label == "Netral":
            if any(x in title.lower() for x in ["gagal", "utang", "tak lolos", "belum layak", "gugur"]):
                sentimen = "Negatif"
            else:
                sentimen = "Positif"  # default anggap positif
        else:
            sentimen = label

        berita_final.append({
            "Tanggal": date,
            "Judul Berita": title,
            "Link": link,
            "Skor Keyakinan": score,
            "Sentimen": sentimen
        })

    except Exception as e:
        print(f"Gagal analisis berita: {title}\nError: {e}\n")

# ======================
# 3. Simpan ke Excel (1 sheet)
# ======================
df = pd.DataFrame(berita_final)
df.to_excel("berita_brks_agustus.xlsx", index=False, engine="openpyxl")

print("📂 Hasil sudah disimpan ke file: berita_brks_agustus.xlsx")
print("Silakan buka file Excel tersebut untuk melihat daftar berita dengan tanggal, sentimen positif/negatif.")


✅ Keyword 'Bank Riau Kepri Syariah' menghasilkan 10 berita
✅ Keyword 'BRK Syariah' menghasilkan 10 berita
✅ Keyword 'Bank Riau Kepri' menghasilkan 10 berita

📊 Total unik berita ditemukan: 18



Device set to use cpu


📂 Hasil sudah disimpan ke file: berita_brks_agustus.xlsx
Silakan buka file Excel tersebut untuk melihat daftar berita dengan tanggal, sentimen positif/negatif.
